# Problem-1 (Question-1) How To Create Dataframes From Wikipedia Tables

In [236]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Use SHIFT+TAB keys to popup inplace code help
%config IPCompleter.greedy = True

# Output multiple statements from one input cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Customize Notebook

**table_from_top.** If the Wikipedia page has one table then use `table_from_top = 1` value. Otherwise count table number from top and replace value to get specific table.

**wikipedia_page.** Specify the wikipedia page name from where to source dataset. The CSV file will be saved with the same name.

**trace.** Set `trace = True` to trace how feature values are extracted. Does not save extracted dataset. Prefixes applied parsing/extraction rules to extracted values.

In [237]:
table_from_top = 1
wikipedia_page = 'List_of_postal_codes_of_Canada:_M'
trace = False

## Load and Parse

This section loads the Wikipedia page and parses the table data we are interested in converting to a dataset.

In [238]:
wikipedia_url = 'https://en.wikipedia.org/wiki/{}'.format(wikipedia_page)
page = requests.get(wikipedia_url)
soup = BeautifulSoup(page.content, 'lxml')
tables = soup.find_all('table', {'class': 'wikitable'})
table = tables[table_from_top - 1]

## Quick Preview

This section extracts the table header with feature or column names.

Use this section to quick preview if you have the right table in processing.

In [239]:
feature_names = []

header_row = table.find('tr')
for header in header_row.find_all('th'):
    feature_name = ' '.join(header.find_all(text=True))    
    feature_names.append(feature_name.replace('\n', ''))
print(feature_names)

['Postcode', 'Borough', 'Neighbourhood']


## Data Wrangling

This section applies data wrangling rules based on exceptions found when parsing Wikipedia tables.

- If a feature value contains a link then extract text from the link.
- Ignore text which starts with `[` square brackets.
- Ignore image links (...flags) prefix link text.
- Ignore hidden text used for IDs.

In [240]:
samples = []
sample_rows = table.find_all('tr')[1:]
for sample_row in sample_rows:
    features = []
    for feature_col in sample_row.find_all('td'):
        feature_value = ''
        text = feature_col.text.replace('\n','')
        features.append(text)
        href_tags = soup.find_all(href=True)
    features.append(feature_value)
    samples.append(dict(zip(feature_names, features)))
#sample_rows

## Preview Dataset

This section enables you to preview the parsed dataset.

In [241]:
df = pd.DataFrame(samples)
#col_list = list(df)
# use this handy way to swap the elements
#col_list[0], col_list[1],  col_list[2] = col_list[2], col_list[0], col_list[1]
# assign back, the order will now be swapped
#df.columns = col_list
final_df = df[["Postcode", "Borough", "Neighbourhood"]]
#final_df
final_df.head()
final_df.tail()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


,Postcode,Borough,Neighbourhood
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
287,M9Z,Not assigned,Not assigned


# Output Data Frame Creation
### Group the Neighbourhood based on the Postal Code

In [242]:
#Group the Neighbourhood based on the Postal Code
output_df = final_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
ordered_df = output_df[['Postcode','Borough', 'Neighbourhood']]

# Assign the Borough value to Neighbourhood value if the Borough has a value and the Neighbourhood has no assigned value

In [243]:
for i, row in ordered_df.iterrows():
    if(row['Borough'] != 'Not assigned' and row['Neighbourhood'] == 'Not assigned'):
        print(row['Borough'])
        row['Neighbourhood'] = row['Borough']

Queen's Park


# Display the final Output

In [244]:
ordered_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
9,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


# Display the shape of the Output

In [245]:
ordered_df.shape

(180, 3)